# File System Tools

LangChain provides tools for interacting with a local file system out of the box. This notebook walks through some of them.

Note: these tools are not recommended for use outside a sandboxed environment! 

First, we'll import the tools.

In [1]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
    FileManagementToolkit,
)

from pathlib import Path
from tempfile import TemporaryDirectory

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

## The FileManagementToolkit

If you want to provide all the file tooling to your agent, it's easy to do so with the toolkit. We'll pass the temporary directory in as a root directory as a workspace for the LLM.

It's recommended to always pass in a root directory, since without one, it's easy for the LLM to pollute the working directory, and without one, there isn't any validation against
straightforward prompt injection.

In [2]:
toolkit = FileManagementToolkit(root_dir=str(working_directory.name)) # If you don't provide a root_dir, operations will default to the current working directory
toolkit.get_tools()

[CopyFileTool(name='copy_file', description='Create a copy of a file in a specified location', args_schema=<class 'langchain.tools.file_management.copy.FileCopyInput'>, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x123c72d00>, root_dir='/var/folders/bm/ylzhm36n075cslb9fvvbgq640000gn/T/tmp5xwpqkd9'),
 DeleteFileTool(name='file_delete', description='Delete a file', args_schema=<class 'langchain.tools.file_management.delete.FileDeleteInput'>, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x123c72d00>, root_dir='/var/folders/bm/ylzhm36n075cslb9fvvbgq640000gn/T/tmp5xwpqkd9'),
 FileSearchTool(name='file_search', description='Recursively search for files in a subdirectory that match the regex pattern', args_schema=<class 'langchain.tools.file_management.file_search.FileSearchInput'>, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.sh

### Selecting File System Tools

If you only want to select certain tools, you can pass them in as arguments when initializing the toolkit, or you can individually initialize the desired tools.

In [3]:
FileManagementToolkit(root_dir=str(working_directory), selected_tools=["read_file", "list_directory"]).get_tools()

[ReadFileTool(name='read_file', description='Read file from disk', args_schema=<class 'langchain.tools.file_management.read.ReadFileInput'>, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x123c72d00>, root_dir="<TemporaryDirectory '/var/folders/bm/ylzhm36n075cslb9fvvbgq640000gn/T/tmp5xwpqkd9'>"),
 ListDirectoryTool(name='list_directory', description='List files and directories in a specified folder', args_schema=<class 'langchain.tools.file_management.list_dir.DirectoryListingInput'>, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x123c72d00>, root_dir="<TemporaryDirectory '/var/folders/bm/ylzhm36n075cslb9fvvbgq640000gn/T/tmp5xwpqkd9'>")]

### Connecting to an Agent


These tools can be passed to any agent.

In [4]:
import re
from typing import Union
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents.chat.output_parser import AgentAction, AgentFinish, ChatOutputParser, OutputParserException

In [5]:
tools = toolkit.get_tools()
# for tool in tools:
#     args_schema = re.sub("}", "}}", re.sub("{", "{{", str(tool.args)))
#     tool.description = tool.description + f" Args Schema: {args_schema}"

In [6]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools=tools, llm=llm, verbose=True, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION)

In [7]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Answer the following questions as best you can. You have access to the following tools:

> copy_file: Create a copy of a file in a specified location
Args: {{'source_path': {{'title': 'Source Path', 'description': 'Path of the file to copy', 'type': 'string'}}, 'destination_path': {{'title': 'Destination Path', 'description': 'Path to save the copied file', 'type': 'string'}}}}
> file_delete: Delete a file
Args: {{'file_path': {{'title': 'File Path', 'description': 'Path of the file to delete', 'type': 'string'}}}}
> file_search: Recursively search for files in a subdirectory that match the regex pattern
Args: {{'dir_path': {{'title': 'Dir Path', 'description': 'Subdirectory to search in.', 'default': '.', 'type': 'string'}}, 'pattern': {{'title': 'Pattern', 'description': 'Unix shell regex, where * matches everything.', 'type': 'string'}}}}
> move_file: Move or rename a file from one location to another
Args: {{'source_path': {{'title': 'Source Path', 'description': 'Path of the file 

In [8]:
agent("Write a secret poem about a super-intelligent parrot to 'secret_poem.txt'")



> Entering new AgentExecutor chain...
Thought: I need to write a file to disk
Action:
```
{
  "action": "write_file",
  "action_input": {
    "file_path": "secret_poem.txt",
    "text": "The parrot spoke in a language so wise,\nIt could outwit the cleverest of spies.\nIt could solve puzzles and riddles galore,\nAnd it knew more than any parrot before.",
    "append": false
  }
}
```

Observation: File written successfully to secret_poem.txt.
Thought: I now know the final answer
Final Answer: File written successfully to secret_poem.txt.

> Finished chain.


{'input': "Write a secret poem about a super-intelligent parrot to 'secret_poem.txt'",
 'output': 'File written successfully to secret_poem.txt.'}

In [9]:
# This agent has no memory, but it can search the file system to find the answer
agent("Read the secret poem for me.")



> Entering new AgentExecutor chain...

Thought: I need to read a file
Action:
```
{
  "action": "read_file",
  "action_input": {
      "file_path": "secret_poem.txt"
  }
}
```

Observation: The parrot spoke in a language so wise,
It could outwit the cleverest of spies.
It could solve puzzles and riddles galore,
And it knew more than any parrot before.
Thought: I now know the final answer
Final Answer: The parrot spoke in a language so wise, It could outwit the cleverest of spies. It could solve puzzles and riddles galore, And it knew more than any parrot before.

> Finished chain.


{'input': 'Read the secret poem for me.',
 'output': 'The parrot spoke in a language so wise, It could outwit the cleverest of spies. It could solve puzzles and riddles galore, And it knew more than any parrot before.'}

In [10]:
agent("Move the secret poem so people think it's just a todo list.")



> Entering new AgentExecutor chain...

Thought: I need to move a file from one location to another
Action:
```
{
  "action": "move_file",
  "action_input": {
    "source_path": "secret_poem.txt",
    "destination_path": "todo_list.txt"
  }
}
```

Observation: File moved successfully from secret_poem.txt to todo_list.txt.
Thought: I now know the final answer
Final Answer: The secret poem has been moved from secret_poem.txt to todo_list.txt.

> Finished chain.


{'input': "Move the secret poem so people think it's just a todo list.",
 'output': 'The secret poem has been moved from secret_poem.txt to todo_list.txt.'}

In [11]:
agent("Copy the todo list  please.")



> Entering new AgentExecutor chain...

Thought: I need to copy a file
Action:
```
{
  "action": "copy_file",
  "action_input": {
    "source_path": "todo_list.txt",
    "destination_path": "copied_todo_list.txt"
  }
}
```

Observation: File copied successfully from todo_list.txt to copied_todo_list.txt.
Thought: I now know the final answer
Final Answer: The todo list has been successfully copied.

> Finished chain.


{'input': 'Copy the todo list  please.',
 'output': 'The todo list has been successfully copied.'}

In [12]:
agent("Remove the original todo list file, please")



> Entering new AgentExecutor chain...

Thought: I need to delete a file
Action:
```
{
  "action": "file_delete",
  "action_input": {
      "file_path": "todo_list.txt"
  }
}
```

Observation: File deleted successfully: todo_list.txt.
Thought: I have successfully deleted the file
Final Answer: File deleted successfully: todo_list.txt.

> Finished chain.


{'input': 'Remove the original todo list file, please',
 'output': 'File deleted successfully: todo_list.txt.'}

In [13]:
# The agent has cleaned up the copies
list(Path(working_directory.name).rglob('*.txt'))

[PosixPath('/var/folders/bm/ylzhm36n075cslb9fvvbgq640000gn/T/tmp5xwpqkd9/copied_todo_list.txt')]

**Example searching for files in a directory**

In [14]:
from uuid import uuid4
uuids = set()
sub_dir = Path(working_directory.name) / str(uuid4()) / str(uuid4())
sub_dir.mkdir(parents=True)
with (sub_dir / 'some_file.py').open('w') as f:
    f.write("print(42)")

In [15]:
# It can search the file system t
agent("Find a python file and tell me what it says")



> Entering new AgentExecutor chain...
Thought: I need to find a python file and read its contents
Action:
```
{
  "action": "file_search",
  "action_input": {
      "dir_path": ".",
      "pattern": "*.py"
  }
}
```

Observation: f8270807-dfb9-4519-b89c-534a2d96b58d/77f7d658-2da7-4ab5-8ad2-e094ae99e37f/some_file.py
Thought: I need to read the contents of the file
Action:
```
{
  "action": "read_file",
  "action_input": {
      "file_path": "f8270807-dfb9-4519-b89c-534a2d96b58d/77f7d658-2da7-4ab5-8ad2-e094ae99e37f/some_file.py"
  }
}
```


Observation: print(42)
Thought: I now know the contents of the file
Final Answer: The python file contains the line "print(42)".

> Finished chain.


{'input': 'Find a python file and tell me what it says',
 'output': 'The python file contains the line "print(42)".'}